In [ ]:
%pip install -q --upgrade langchain langgraph langchain_openai langchain_core langgraph python-dotenv

#### Exemplo de agente simples criado usando LangGraph, LangChain e o modelo Llama-4 chamado via OpenRouter

Cria o nosso LLM usando o modelo escolhido

In [ ]:
import os
from typing import Optional

from dotenv import load_dotenv
from langchain_core.utils.utils import secret_from_env
from langchain_openai import ChatOpenAI
from pydantic import Field, SecretStr

load_dotenv()


class ChatOpenRouter(ChatOpenAI):
    openai_api_key: Optional[SecretStr] = Field(
        alias="api_key", default_factory=secret_from_env("OPENROUTER_API_KEY", default=None)
    )

    @property
    def lc_secrets(self) -> dict[str, str]:
        return {"openai_api_key": "OPENROUTER_API_KEY"}

    def __init__(self, openai_api_key: Optional[str] = None, **kwargs):
        openai_api_key = openai_api_key or os.environ.get("OPENROUTER_API_KEY")
        super().__init__(base_url="https://openrouter.ai/api/v1", openai_api_key=openai_api_key, **kwargs)


llm = ChatOpenRouter(model_name="meta-llama/llama-4-maverick:free")

Define a função que será o node principal do nosso assistente e a mensagem que da o contexto que o agente deve seguir

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import MessagesState

sys_msg = SystemMessage(content="You are a helpful assistant to respond to general purpose queries.")


def assistant(state: MessagesState):
    return {"messages": [llm.invoke([sys_msg] + state["messages"])]}

Cria o grafo que representa o comportamento do nosso agente, adicionando os estados iniciais e finais ligados ao node do agente

In [ ]:
from IPython.display import Image
from langgraph.graph import END, START, StateGraph

builder = StateGraph(MessagesState)

builder.add_node("assistant", assistant)

builder.add_edge(START, "assistant")
builder.add_edge("assistant", END)

react_graph = builder.compile()

display(Image(react_graph.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
messages = [HumanMessage(content="Tell me, what is the smallest planet in the solar system?")]
messages = react_graph.invoke({"messages": messages})

for m in messages["messages"]:
    m.pretty_print()